In [1]:
from pymongo import MongoClient

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('datamad1019','restaurants')

In [9]:
rest = list(coll.find({'name':"Wendy'S"}))

In [11]:
print(len(rest))

41


In [20]:
rest[0]['address']['coord']

[{'$numberDouble': '-73.961704'}, {'$numberDouble': '40.662942'}]

## GeoJSON

{ type: "Point", coordinates: [ 40, 5 ] }

In [25]:
def getLocation(wendy):
    longitude = wendy['address']['coord'][0]['$numberDouble']
    latitude = wendy['address']['coord'][1]['$numberDouble']
    loc = {
        'type':'Point',
        'coordinates':[float(longitude), float(latitude)]
    }
    return loc

In [29]:
getLocation(rest[0])

{'type': 'Point', 'coordinates': [-73.961704, 40.662942]}

In [34]:
for wendy in rest:
    value = {"$set": {'location':getLocation(wendy)}}
    coll.update_one(wendy,value)
    # <collection>.update_one(<document>,<value>)

In [38]:
for wendy in rest:
    value = {"$unset": {'address.coord':''}}
    coll.update_one(wendy,value)
    # <collection>.update_one(<document>,<value>)

In [39]:
rest = list(coll.find({'name':"Wendy'S"}))
rest[0]

{'_id': ObjectId('5dcd89ad723a472b365091eb'),
 'address': {'building': '469',
  'street': 'Flatbush Avenue',
  'zipcode': '11225'},
 'borough': 'Brooklyn',
 'cuisine': 'Hamburgers',
 'grades': [{'date': datetime.datetime(2014, 12, 30, 0, 0),
   'grade': 'A',
   'score': 8},
  {'date': datetime.datetime(2014, 7, 1, 0, 0), 'grade': 'B', 'score': 23},
  {'date': datetime.datetime(2013, 4, 30, 0, 0), 'grade': 'A', 'score': 12},
  {'date': datetime.datetime(2012, 5, 8, 0, 0), 'grade': 'A', 'score': 12}],
 'name': "Wendy'S",
 'restaurant_id': '30112340',
 'location': {'type': 'Point', 'coordinates': [-73.961704, 40.662942]}}

In [52]:
manhattanwendys = coll.find(
    {"location":{"$near":
               {"$geometry":{
                   "type":"Point",
                   "coordinates":[-73.9712,40.7831]
               },
                "$maxDistance":10000,
                "$minDistance":5000
               }
               }
    }
)

In [53]:
manhattanwendys=list(manhattanwendys)
len(manhattanwendys)

9